Data story using generalized functions

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
from plotnine import *

import pipeline.sql as plsql
import eda.functions_datastory as funsds

In [ ]:
engine = plsql.create_engine('../config.yaml')
role = 'direccion_trabajo_inspections_write'

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, sum(cast(urgencia as int)) as urgents, 
    sum(derechofund) as humanrights,
    sum(infra) as infractions, count(*) as inspections
    from raw.inspections_complete
    where ccae = '133000'
    group by rutempresamask;"""
#tab_ins = plsql.query(qry, engine)
#id_company = ''.join(tab_ins[tab_ins['urgents']>45].rutempresamask)

id_company = 'ff11b3633e758a918f6a1a23425ef3304b724706aed0b2898b887a1c0b11f565'
print (id_company)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, avg(emptrabhombres) as workers, count(*) as inspections, sum(infra) as infractions
    from raw.inspections_complete
    where ccae = '133000'
    group by rutempresamask;"""
tab = plsql.query(qry, engine)
tab['ind'] = tab['inspections'] > 200

In [ ]:
(ggplot(tab[pd.isnull(tab['infractions']) == False], 
        aes('inspections', 'infractions')) + 
    geom_point(aes( color = 'ind'), size = 2, 
              alpha = .9) + 
    scale_color_manual(values = ("black", '#e91d63'),
                   guide = False) + 
    geom_abline(intercept = 0, slope = 1, color = "#e91d63") +
    theme(figure_size = (4, 3)))

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from raw.inspections_complete
    limit 3;"""
df = plsql.query(qry, engine)
df.columns

In [ ]:
funsds.facility_info(engine = engine, id_company = id_company)

In [ ]:
funsds.time_graphs(engine = engine, id_company = id_company)

In [ ]:
tab_result = funsds.freq_matters_bookupdated(engine, role, id_company)
tab_result.head()

In [ ]:
funsds.text_wordcloud_matters(tab_result)